## Setup

In [1]:
!export PYTHONPATH=$PYTHONPATH:"drive/MyDrive/OML_project"

In [2]:
%cd "drive/MyDrive/OML_project"

/content/drive/MyDrive/OML_project


In [3]:
!nvidia-smi

Sun Jun  5 23:30:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Training

Change only **config_filename** variable!

In [ ]:
import argparse
import os
import json
import torch
from utils.network import build_network
from utils.dataset import build_dataset
from utils.train import train
from utils.utils import setup
from torch.utils.tensorboard import SummaryWriter


# CHANGE ONLY THIS PARAMETER
config_filename = "config1"

# Create configs, logs and checkpoints folder
if not os.path.exists("configs/"):
    os.mkdir("configs/")
if not os.path.exists("logs/"):
    os.mkdir("logs/")
if not os.path.exists("checkpoints/"):
    os.mkdir("checkpoints/")

config_path = 'configs/' + config_filename + '.json'
# Get config
with open(config_path) as json_config:
    config = json.load(json_config)

config["name"] = config_filename

# Set folder for logging
log_dir = config["log_dir"] + '/' + config["name"]
if not os.path.exists(log_dir):
    os.mkdir(log_dir)

# Set folder for checkpoints
checkpoints_dir = config["checkpoint_dir"] + '/' + config["name"]
if not os.path.exists(checkpoints_dir):
    os.mkdir(checkpoints_dir)

config["checkpoint_dir"] = checkpoints_dir
config["log_dir"] = log_dir

# Check for cuda availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device state: ', device)

# Get dataset
dataset = build_dataset(config, config["data"], train=True)

# Get network
net = build_network(config)
net.to(device=device)

# Train network
writer = SummaryWriter(log_dir=config["log_dir"])
train(dataset, net, config, writer, device=device)


## Use Tensorboard

Change log folder (logs/\<change_this\>) to the desired one!

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=logs/config1

## Testing

Change only **config_filename** variable! Also, you can change **image_indexes** to a specific list of indexes if you want to test only some images.

In [ ]:
import os
import json
import torch
from utils.train import predict
from utils.network import build_network
from utils.dataset import build_dataset

# CHANGE ONLY THIS PARAMETER
config_filename = "config1"

# Create configs, logs and checkpoints folder
if not os.path.exists("configs/"):
    os.mkdir("configs/")
if not os.path.exists("logs/"):
    os.mkdir("logs/")
if not os.path.exists("checkpoints/"):
    os.mkdir("checkpoints/")

config_path = 'configs/' + config_filename + '.json'
# Get config
with open(config_path) as json_config:
    config = json.load(json_config)

config["name"] = config_filename

# Set folder for logging
log_dir = config["log_dir"] + '/' + config["name"]
if not os.path.exists(log_dir):
    os.mkdir(log_dir)

# Set folder for checkpoints
checkpoints_dir = config["checkpoint_dir"] + '/' + config["name"]
if not os.path.exists(checkpoints_dir):
    os.mkdir(checkpoints_dir)

config["checkpoint_dir"] = checkpoints_dir
config["log_dir"] = log_dir

# Check for cuda availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device state: ', device)

# Get network
net = build_network(config)
net.to(device=device)
model_path = config["checkpoint_dir"] + '/' + 'bestmodel.pth'
net.load_state_dict(torch.load(model_path, map_location=device))

# Get an image
test_dataset = build_dataset(config, config["data"])

# Make predictions
# image_indexes = list(range(100))
image_indexes = None
predict(test_dataset, net, device, image_indexes)